In [114]:
import numpy as np
import time

In [3]:
nSamples = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [4]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# Generate

In [5]:
for x in n:
    time = 0
    fileData = []
    fileData.append(x)
    for _ in range(x):
        p = np.random.randint(1, 10)
        r = np.random.randint(time - 5 if time - 5 > 0 else 0 ,time + 5)
        d = np.random.randint(r+p, r+p+10)
        w = np.random.randint(0,10)
        time += p
        
        fileData.append([p,r,d,w])
    with open(f'data/instances_{x}.csv', "w") as f:
        f.write(f"{str(fileData[0])}\n")
        for line in fileData[1:]:            
            f.write(f"{' '.join([str(x) for x in line])}\n")

NameError: name 'n' is not defined

# Verify

In [64]:
x = 50

In [129]:
motherFolder = "out" # "PTSZ-instancje"
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
#    print(index)
    for samples in nSamples:
        with open(f'{motherFolder}/{index}/out{index}_{samples}.txt', "r") as result:
            res = result.read().split('\n')
            weight = int(res[0])
            gen = (t for t in res[1].split(" ") if t.isdigit())
            taskQueue = [int(t) for t in gen]

        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])                
            for i,line in enumerate(d[1:]):
                if line == '':
                    continue
                p,r,d,w = line.split(' ')
                p,r,d,w = int(p), int(r), int(d), int(w)
                data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0 }
                    
        calculatedWeight = 0
        timeCounter = 0
        for task in taskQueue:
            if timeCounter < data[task]['r']:
                timeCounter = data[task]['r']
            # add p time to current time
            timeCounter += data[task]['p']
            c = timeCounter
            data[task]['c'] = c

            # check correctness
            if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
                print("Error")

            # verify cost
            if data[task]["c"] > data[task]["d"]:
                calculatedWeight += data[task]["w"]
        print(calculatedWeight)
        #if calculatedWeight != weight:
        #    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
        #else:
        #    print(f"Weight: {calculatedWeight} confirmed")
    #print()
    #print()
        
        

82
220
419
426
561
680
806
976
1174
1180
97
423
1014
1901
2783
4170
5853
8048
10243
11948
217
927
2201
3849
5803
8861
11955
16431
20297
24872
242
533
842
1148
1363
1653
1857
2187
2494
2792
0
38
774
1530
2075
2555
3086
3690
4208
4967
86
416
1076
2056
2915
4016
5977
7528
10473
12042
163
605
1263
2130
3455
4813
6397
7732
10208
13372
245
491
755
1039
1265
1539
1723
2054
2241
2468
88
145
287
331
546
566
556
869
734
1007
244
510
717
929
1192
1522
1713
2061
2187
2420
937
1653
2726
4038
4789
5451
6544
7496
8785
10634
0
0
0
0
0
0
0
0
0
0
1902
3887
5872
7894
9772
12017
13899
15638
17991
19600
189
341
1066
1326
1776
2056
3611
4056
4708
5191


In [ ]:
# p - persistance
# r - ready time
# d - oczekiwany termin zakończenia
# w - weight
# Cj oznacza moment zakończenia wykonywania zadania Jj w uszeregowaniu

# p r d w
# r  >= c - p

# TaskScheduler

In [128]:
data = {}
indicies = ["136805", "136792", "136683", "132231", "136730", "136682", "136764", "136782", "136723", "136778", "136309", "136718", "136315", "136759"]
for index  in indicies:
    for samples in nSamples:
        with open(f'PTSZ-instancje/{index}/in{index}_{samples}.txt', "r") as instance:
            d = instance.read().split('\n')
            n = int(d[0])                
            for i,line in enumerate(d[1:]):
                if line == '':
                    continue
                p,r,d,w = line.split(' ')
                p,r,d,w = int(p), int(r), int(d), int(w)
                data[i+1] = {"p": p, "r": r, "d": d, "w": w, "c": 0, "penalty": 0 }
                    
        calculatedWeight = 0
        timeCounter = 0
        outSchedule = []
        
        start = time.time()
        while len(data) > 0:
            readyToGo = {}
            for key, task in data.items():
                if task['r'] <= timeCounter: # add penalty ( (timeCounter+p-d)*w )
                    endTime = timeCounter+task['p']-task['d']
                    task['penalty'] = ( endTime if endTime > 0 else 0 ) * task['w']
                    readyToGo[key] = task
                
            if len(readyToGo) == 0: # currently there is no task to go
                #print("upsi")
                timeCounter += 1
                continue
            #print(f"readyToGo: {readyToGo}")

            readyToGoSorted = sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['w']), reverse=True)
            selectedTask = readyToGoSorted.pop(0)
            #print(timeCounter)
            #print(f"selectedTask: {selectedTask}")

            data.pop(selectedTask[0])
            timeCounter += selectedTask[1]['p']
            outSchedule.append(selectedTask[0])    
        
        end = time.time()
        #print(end-start)
        with open(f'out/{index}/out{index}_{samples}.txt', "w") as f:
            f.write(f"{samples}\n")         
                #f.write(f"{' '.join([str(x) for x in line])}\n")
            f.write(f"{' '.join([str(x) for x in outSchedule])}\n")
        
    

In [62]:
 sorted(readyToGo.items(), key=lambda y: (y[1]['penalty'], y[1]['d']), reverse=True).pop(0)[0]

1

In [102]:
for dirc in indicies:
    try:
        os.mkdir(f"out/{dirc}")
    except:
        pass

In [56]:
readyToGo.items()

dict_items([(1, {'p': 7, 'r': 1, 'd': 17, 'w': 9, 'c': 0, 'penalty': 45}), (2, {'p': 9, 'r': 7, 'd': 23, 'w': 3, 'c': 0, 'penalty': 3}), (23, {'p': 4, 'r': 4, 'd': 15, 'w': 1, 'c': 0, 'penalty': 4})])

In [9]:
calculatedWeight = 0
time = 0
for task in taskQueue:
    if time < data[task]['r']:
        time = data[task]['r']
    # add p time to current time
    time += data[task]['p']
    c = time
    data[task]['c'] = c
    
    # check correctness
    if not data[task]["r"] <= data[task]["c"] - data[task]["p"]: 
        print("Error")
    
    # verify cost
    if data[task]["c"] > data[task]["d"]:
        calculatedWeight += data[task]["w"]
if calculatedWeight != weight:
    print(f"wrong weight. Calculated: {calculatedWeight}, read: {weight}")
else:
    print(f"Weight: {calculatedWeight} confirmed")

Weight: 0 confirmed
